# AXON: Examples

In [1]:
from axon.api import loads, dumps
from axon.objects import Node, node
from axon.objects import Builder, register_builder
from axon import dump_as_str, as_unicode, factory, reduce

from datetime import date, time, datetime
from pprint import pprint
from axon.errors import AxonError

try:
    from cdecimal import Decimal  
except:
    from decimal import Decimal

## Simple values

### Integer values

In [2]:
vs = loads('0 1 -1 32768 -9223372036854775807')
pprint(vs)
print(dumps(vs))

[0, 1, -1, 32768, -9223372036854775807]
0
1
-1
32768
-9223372036854775807


In [3]:
vs = loads('340282366920938463463374607431768211456 -340282366920938463463374607431768211456') # long int value =/-2**128
pprint(vs)
print(dumps(vs))

[340282366920938463463374607431768211456,
 -340282366920938463463374607431768211456]
340282366920938463463374607431768211456
-340282366920938463463374607431768211456


### Float values

In [4]:
vs = loads('0. 0.0 1.0 -1.0 -0.0 3.141528 1.41e2 -1.41e-2')
pprint(vs)
print(dumps(vs))

[0.0, 0.0, 1.0, -1.0, -0.0, 3.141528, 141.0, -0.0141]
0.0
0.0
1.0
-1.0
-0.0
3.141528
141.0
-0.0141


In [5]:
try:
    vs = loads('.12')
except AxonError:
    print('invalid float number')

invalid float number


In [6]:
vs = loads('$NaN ∞ -∞ ?')
pprint(vs)
print(dumps(vs))

[nan, inf, -inf, nan]
?
∞
-∞
?


In [7]:
try:
    vs = loads('.')
except AxonError:
    print('invalid float number')

invalid float number


### Decimal values

In [8]:
vs = loads('0D 1d -1D 0.d 0.0d 1.0D -1.0D -0.0D 3.141528D 1.41e2d -1.41e-2d')
pprint(vs)
print(dumps(vs))

[Decimal('0'),
 Decimal('1'),
 Decimal('-1'),
 Decimal('0'),
 Decimal('0.0'),
 Decimal('1.0'),
 Decimal('-1.0'),
 Decimal('-0.0'),
 Decimal('3.141528'),
 Decimal('141'),
 Decimal('-0.0141')]
0D
1D
-1D
0D
0.0D
1.0D
-1.0D
-0.0D
3.141528D
141D
-0.0141D


In [9]:
vs = loads('$NaND ∞$ -∞$ ?$')
pprint(vs)
print(dumps(vs))

[Decimal('NaN'), Decimal('Infinity'), Decimal('-Infinity'), Decimal('NaN')]
?D
∞D
-∞D
?D


### Unicode text

In [10]:
vs = loads('''"any unicode text
with line breaks
and control symbols \t\n"''')
pprint(vs)
print(dumps(vs))

['any unicode text\nwith line breaks\nand control symbols \t\n']
"any unicode text
with line breaks
and control symbols 	
"


In [11]:
vs = loads('''"text line \
splited \
into chunks"''')
pprint(vs)
print(dumps(vs))

['text line splited into chunks']
"text line splited into chunks"


### Date/Time values

In [12]:
vs = loads('''2012-12-31 12:30 12:30:59 12:30:59.150000 12:30:59+03 2012-12-31T12:30:59.015000-04''')
pprint(vs)
print(dumps(vs, pretty=1))

[datetime.date(2012, 12, 31),
 datetime.time(12, 30),
 datetime.time(12, 30, 59),
 datetime.time(12, 30, 59, 150000),
 datetime.time(12, 30, 59, tzinfo=datetime.timezone(datetime.timedelta(0, 10800))),
 datetime.datetime(2012, 12, 31, 12, 30, 59, 15000, tzinfo=datetime.timezone(datetime.timedelta(-1, 72000)))]
2012-12-31
12:30
12:30:59
12:30:59.150000
12:30:59+03
2012-12-31T12:30:59.015000-04


## Anonymous complex objects

### Simply lines of data

In [13]:
text = '''
# No Sym Flag Date Time
[1 "a" $true 2007-12-20 9:00]
[2 "b" $false 2007-12-20 15:00]
[3 "c" $false 2007-12-20 00:00]
'''
vs = loads(text)
pprint(vs)

[[1, 'a', True, datetime.date(2007, 12, 20), datetime.time(9, 0)],
 [2, 'b', False, datetime.date(2007, 12, 20), datetime.time(15, 0)],
 [3, 'c', False, datetime.date(2007, 12, 20), datetime.time(0, 0)]]


In [14]:
print(dumps(vs))

[1 "a" $true 2007-12-20 09:00]
[2 "b" $false 2007-12-20 15:00]
[3 "c" $false 2007-12-20 00:00]


In [15]:
print(dumps(vs, pretty=1, hsize=3))

[ 1 "a" $true
  2007-12-20
  09:00]
[ 2 "b" $false
  2007-12-20
  15:00]
[ 3 "c" $false
  2007-12-20
  00:00]


### Simply dicts

In [16]:
text = u'''
{"a":1 "b":2.0 "c":3000$}
{"a":100 "b":200.5 "c":30$ "d":$null}
'''
vs = loads(text)
pprint(vs)

[{'a': 1, 'b': 2.0, 'c': Decimal('3000')},
 {'a': 100, 'b': 200.5, 'c': Decimal('30'), 'd': None}]


In [17]:
print(dumps(vs))

{a:1 b:2.0 c:3000D}
{a:100 b:200.5 c:30D d:$null}


In [18]:
print(dumps(vs, pretty=1))

{ a: 1
  b: 2.0
  c: 3000D}
{ a: 100
  b: 200.5
  c: 30D
  d: $null}


### Simply tuples

In [19]:
text = u'''
(2013-01-01 12:30) 
(2013-01-02 13:00)
(2013-01-03 14:30)
'''
values = loads(text)
pprint(values)

[(datetime.date(2013, 1, 1), datetime.time(12, 30)),
 (datetime.date(2013, 1, 2), datetime.time(13, 0)),
 (datetime.date(2013, 1, 3), datetime.time(14, 30))]


In [20]:
print(dumps(values))

(2013-01-01 12:30)
(2013-01-02 13:00)
(2013-01-03 14:30)


In [21]:
print(dumps(values, pretty=1, hsize=2))

(2013-01-01 12:30)
(2013-01-02 13:00)
(2013-01-03 14:30)


### Simply dicts of lists

In [22]:
text = u'''
{ a: [0 1 2] 
  b: [0. 1.0 2.0] 
  c: [0.D 1.0D] 
  d: [$true] }
{ a: [100 200 300] 
  b: [100. 200. 300.] 
  c: [10D 20D] 
  s: [$false] }
'''
values = loads(text)
pprint(values)

[{'a': [0, 1, 2],
  'b': [0.0, 1.0, 2.0],
  'c': [Decimal('0'), Decimal('1.0')],
  'd': [True]},
 {'a': [100, 200, 300],
  'b': [100.0, 200.0, 300.0],
  'c': [Decimal('10'), Decimal('20')],
  's': [False]}]


In [23]:
print(dumps(values))

{a:[0 1 2] b:[0.0 1.0 2.0] c:[0D 1.0D] d:[$true]}
{a:[100 200 300] b:[100.0 200.0 300.0] c:[10D 20D] s:[$false]}


In [24]:
print(dumps(values, pretty=1))

{ a: [0 1 2]
  b: [0.0 1.0 2.0]
  c: [0D 1.0D]
  d: [$true]}
{ a: [100 200 300]
  b: [100.0 200.0 300.0]
  c: [10D 20D]
  s: [$false]}


In [25]:
print(dumps(values, pretty=1, hsize=3))

{ a: [0 1 2]
  b: [0.0 1.0 2.0]
  c: [0D 1.0D]
  d: [$true]}
{ a: [100 200 300]
  b: [100.0 200.0 300.0]
  c: [10D 20D]
  s: [$false]}


## Named complex objects

### Simply named dicts of tuples

In [26]:
text = u'''
rectangle {
    left_top: (-1.0 1.0)
    left_bottom: (-1.0 -1.0)
    right_top: (1.0 1.0)
    right_bottom: (1.0 -1.0)
}
rectangle {
    left_top: (-2.0 2.0)
    left_bottom: (-2.0 -1.0)
    right_top: (1.0 2.0)
    right_bottom: (1.0 -1.0)
}
'''
values = loads(text)
pprint(values)

[rectangle{left_top:(-1.0, 1.0) left_bottom:(-1.0, -1.0) right_top:(1.0, 1.0) right_bottom:(1.0, -1.0)},
 rectangle{left_top:(-2.0, 2.0) left_bottom:(-2.0, -1.0) right_top:(1.0, 2.0) right_bottom:(1.0, -1.0)}]


In [27]:
v1, v2 = values
print(v1)
print(dir(v1))
print(v2)
print(v1.__tag__, v1.__attrs__)
print(v1.left_bottom, v1.right_top)

rectangle{left_top:(-1.0, 1.0) left_bottom:(-1.0, -1.0) right_top:(1.0, 1.0) right_bottom:(1.0, -1.0)}
['__attrs__', '__bool__', '__class__', '__delattr__', '__delitem__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattr__', '__getattribute__', '__getitem__', '__gt__', '__hash__', '__init__', '__iter__', '__le__', '__lt__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__setitem__', '__sizeof__', '__str__', '__subclasshook__', '__tag__', '__vals__']
rectangle{left_top:(-2.0, 2.0) left_bottom:(-2.0, -1.0) right_top:(1.0, 2.0) right_bottom:(1.0, -1.0)}
rectangle OrderedDict([('left_top', (-1.0, 1.0)), ('left_bottom', (-1.0, -1.0)), ('right_top', (1.0, 1.0)), ('right_bottom', (1.0, -1.0))])
(-1.0, -1.0) (1.0, 1.0)


In [28]:
print(dumps(values))

rectangle{left_top:(-1.0 1.0) left_bottom:(-1.0 -1.0) right_top:(1.0 1.0) right_bottom:(1.0 -1.0)}
rectangle{left_top:(-2.0 2.0) left_bottom:(-2.0 -1.0) right_top:(1.0 2.0) right_bottom:(1.0 -1.0)}


In [29]:
print(dumps(values, pretty=1, hsize=2))

rectangle:
  left_top: (-1.0 1.0)
  left_bottom: (-1.0 -1.0)
  right_top: (1.0 1.0)
  right_bottom: (1.0 -1.0)
rectangle:
  left_top: (-2.0 2.0)
  left_bottom: (-2.0 -1.0)
  right_top: (1.0 2.0)
  right_bottom: (1.0 -1.0)


In [30]:
text2 = dumps(values, pretty=2, hsize=2)
print(text2)

rectangle:
  left_top: (-1.0 1.0)
  left_bottom: (-1.0 -1.0)
  right_top: (1.0 1.0)
  right_bottom: (1.0 -1.0)
rectangle:
  left_top: (-2.0 2.0)
  left_bottom: (-2.0 -1.0)
  right_top: (1.0 2.0)
  right_bottom: (1.0 -1.0)


### Simply named lists of tuples

In [31]:
text = u'''
rectangle {
    (-1.0 1.0) 
    (-1.0 -1.0)
    (1.0 -1.0)
    (1.0 1.0)
}
rectangle {
    (-2.0 2.0)
    (-2.0 -1.0)
    (1.0 -1.0)
    (1.0 2.0)
}
'''
values = loads(text)
pprint(values)

[rectangle{(-1.0, 1.0) (-1.0, -1.0) (1.0, -1.0) (1.0, 1.0)},
 rectangle{(-2.0, 2.0) (-2.0, -1.0) (1.0, -1.0) (1.0, 2.0)}]


In [32]:
print(values[0])
print(values[1])
print(values[0].__tag__, values[0].__vals__)

rectangle{(-1.0, 1.0) (-1.0, -1.0) (1.0, -1.0) (1.0, 1.0)}
rectangle{(-2.0, 2.0) (-2.0, -1.0) (1.0, -1.0) (1.0, 2.0)}
rectangle [(-1.0, 1.0), (-1.0, -1.0), (1.0, -1.0), (1.0, 1.0)]


In [33]:
print(dumps(values))

rectangle{(-1.0 1.0) (-1.0 -1.0) (1.0 -1.0) (1.0 1.0)}
rectangle{(-2.0 2.0) (-2.0 -1.0) (1.0 -1.0) (1.0 2.0)}


In [34]:
print(dumps(values, pretty=1))

rectangle:
  (-1.0 1.0)
  (-1.0 -1.0)
  (1.0 -1.0)
  (1.0 1.0)
rectangle:
  (-2.0 2.0)
  (-2.0 -1.0)
  (1.0 -1.0)
  (1.0 2.0)


In [35]:
print(dumps(values, pretty=1, hsize=2))

rectangle:
  (-1.0 1.0)
  (-1.0 -1.0)
  (1.0 -1.0)
  (1.0 1.0)
rectangle:
  (-2.0 2.0)
  (-2.0 -1.0)
  (1.0 -1.0)
  (1.0 2.0)


In [36]:
print(dumps(values, pretty=2, hsize=2))

rectangle:
  (-1.0 1.0)
  (-1.0 -1.0)
  (1.0 -1.0)
  (1.0 1.0)
rectangle:
  (-2.0 2.0)
  (-2.0 -1.0)
  (1.0 -1.0)
  (1.0 2.0)


### Simple file system: attributed tree

In [37]:
text = u'''
folder {
  name: "/"
  folder {
    name: "Library"
    file {name: "book.pdf"}
    file {name: "README"}
    folder {
      name: "Temp"
    }
  }
  folder {
    name: "Desktop"
    file {name: "Notes.txt"}
    file {name: "Recipes.txt"}
  }
}
'''
values = loads(text)
pprint(values)

[folder{name:'/'folder{name:'Library'file{name:'book.pdf'} file{name:'README'} folder{name:'Temp'}} folder{name:'Desktop'file{name:'Notes.txt'} file{name:'Recipes.txt'}}}]


In [38]:
val = values[0]
print(val.__tag__)
print(val[0].__tag__, val[1].__tag__)
print(val[0][0].__tag__, val[1][0].__tag__)
print(val[1].__tag__, val[1].__attrs__, val[1].__vals__)

folder
folder folder
file file
folder OrderedDict([('name', 'Desktop')]) [file{name:'Notes.txt'}, file{name:'Recipes.txt'}]


In [39]:
print(dumps(values))

folder{name:"/" folder{name:"Library" file{name:"book.pdf"} file{name:"README"} folder{name:"Temp"}} folder{name:"Desktop" file{name:"Notes.txt"} file{name:"Recipes.txt"}}}


In [40]:
text2 = dumps(values, pretty=1)
print(text2)

folder:
  name: "/"
  folder:
    name: "Library"
    file:
      name: "book.pdf"
    file:
      name: "README"
    folder:
      name: "Temp"
  folder:
    name: "Desktop"
    file:
      name: "Notes.txt"
    file:
      name: "Recipes.txt"


In [41]:
values2 = loads(text2)
print(dumps(values, pretty=2))

folder:
  name: "/"
  folder:
    name: "Library"
    file:
      name: "book.pdf"
    file:
      name: "README"
    folder:
      name: "Temp"
  folder:
    name: "Desktop"
    file:
      name: "Notes.txt"
    file:
      name: "Recipes.txt"


### Network of connected units

In [42]:
text = u'''
network {
    nodes {
        &x1 node {label:"x1"}
        &x2 node {label:"x2"}
        &x3 node {label:"x3"}
        &u1 node {label:"u1"}
        &u2 node {label:"u2"}
        &y node {label:"y"}
    }
    units {
        unit {out:(*u1) inp:(*x1 *x2 *x3) weights:(1.0 0.5 0.5)}
        unit {out:(*u2) inp:(*x1 *x2) weights:(1.0 1.5)}
        unit {out:(*y) inp:(*u1 *u2) weights:(0.5 -1.0)}
    }
}
'''
values = loads(text)
pprint(values)

[network{nodes{node{label:'x1'} node{label:'x2'} node{label:'x3'} node{label:'u1'} node{label:'u2'} node{label:'y'}} units{unit{out:(node{label:'u1'},) inp:(node{label:'x1'}, node{label:'x2'}, node{label:'x3'}) weights:(1.0, 0.5, 0.5)} unit{out:(node{label:'u2'},) inp:(node{label:'x1'}, node{label:'x2'}) weights:(1.0, 1.5)} unit{out:(node{label:'y'},) inp:(node{label:'u1'}, node{label:'u2'}) weights:(0.5, -1.0)}}}]


In [43]:
val = values[0]
print('Same objects here:')
print(val[0][0] is val[1][0].inp[0])
print(val[0][-1] is val[1][-1].out[0])

Same objects here:
True
True


In [44]:
print(dumps(values))

network{nodes{node{label:"x1"} node{label:"x2"} node{label:"x3"} node{label:"u1"} node{label:"u2"} node{label:"y"}} units{unit{out:(node{label:"u1"}) inp:(node{label:"x1"} node{label:"x2"} node{label:"x3"}) weights:(1.0 0.5 0.5)} unit{out:(node{label:"u2"}) inp:(node{label:"x1"} node{label:"x2"}) weights:(1.0 1.5)} unit{out:(node{label:"y"}) inp:(node{label:"u1"} node{label:"u2"}) weights:(0.5 -1.0)}}}


In [45]:
print(dumps(values, crossref=1))

network{nodes{&1 node{label:"x1"} &2 node{label:"x2"} &3 node{label:"x3"} &5 node{label:"u1"} &4 node{label:"u2"} &6 node{label:"y"}} units{unit{out:(*5) inp:(*1 *2 *3) weights:(1.0 0.5 0.5)} unit{out:(*4) inp:(*1 *2) weights:(1.0 1.5)} unit{out:(*6) inp:(*5 *4) weights:(0.5 -1.0)}}}


In [46]:
print(dumps(values, crossref=1, pretty=1, hsize=3))

network:
  nodes:
    &1 node:
      label: "x1"
    &2 node:
      label: "x2"
    &3 node:
      label: "x3"
    &5 node:
      label: "u1"
    &4 node:
      label: "u2"
    &6 node:
      label: "y"
  units:
    unit:
      out: (*5)
      inp: (*1 *2 *3)
      weights: (1.0 0.5 0.5)
    unit:
      out: (*4)
      inp: (*1 *2)
      weights: (1.0 1.5)
    unit:
      out: (*6)
      inp: (*5 *4)
      weights: (0.5 -1.0)


In [47]:
print(dumps(values, crossref=1, pretty=2, hsize=3))

network:
  nodes:
    &1 node:
      label: "x1"
    &2 node:
      label: "x2"
    &3 node:
      label: "x3"
    &5 node:
      label: "u1"
    &4 node:
      label: "u2"
    &6 node:
      label: "y"
  units:
    unit:
      out: (*5)
      inp: (*1 *2 *3)
      weights: (1.0 0.5 0.5)
    unit:
      out: (*4)
      inp: (*1 *2)
      weights: (1.0 1.5)
    unit:
      out: (*6)
      inp: (*5 *4)
      weights: (0.5 -1.0)


### XML-style representation with collections of elements with same name

In [48]:
text = u'''
network {
    nodes {
        &x1 node {label:"x1"}
        &x2 node {label:"x2"}
        &x3 node {label:"x3"}
        &u1 node {label:"u1"}
        &u2 node {label:"u2"}
        &y  node {label:"y"}
    }
    units {
        unit {out:(*u1) inp:(*x1 *x2 *x3) weights:(1.0 0.5 0.5)}
        unit {out:(*u2) inp:(*x1 *x2) weights:(1.0 1.5)}
        unit {out:(*y) inp:(*u1 *u2) weights:(0.5 -1.0)}
    }
}
'''
values = loads(text)
pprint(values)

[network{nodes{node{label:'x1'} node{label:'x2'} node{label:'x3'} node{label:'u1'} node{label:'u2'} node{label:'y'}} units{unit{out:(node{label:'u1'},) inp:(node{label:'x1'}, node{label:'x2'}, node{label:'x3'}) weights:(1.0, 0.5, 0.5)} unit{out:(node{label:'u2'},) inp:(node{label:'x1'}, node{label:'x2'}) weights:(1.0, 1.5)} unit{out:(node{label:'y'},) inp:(node{label:'u1'}, node{label:'u2'}) weights:(0.5, -1.0)}}}]


In [49]:
val = values[0]
print(val[0][0] is val[1][0].inp[0])
print(val[0][-1] is val[1][-1].out[0])

True
True


In [50]:
print(dumps(values, crossref=1))

network{nodes{&3 node{label:"x1"} &4 node{label:"x2"} &1 node{label:"x3"} &2 node{label:"u1"} &6 node{label:"u2"} &5 node{label:"y"}} units{unit{out:(*2) inp:(*3 *4 *1) weights:(1.0 0.5 0.5)} unit{out:(*6) inp:(*3 *4) weights:(1.0 1.5)} unit{out:(*5) inp:(*2 *6) weights:(0.5 -1.0)}}}


In [51]:
print(dumps(values, crossref=1, pretty=1, hsize=3))

network:
  nodes:
    &3 node:
      label: "x1"
    &4 node:
      label: "x2"
    &1 node:
      label: "x3"
    &2 node:
      label: "u1"
    &6 node:
      label: "u2"
    &5 node:
      label: "y"
  units:
    unit:
      out: (*2)
      inp: (*3 *4 *1)
      weights: (1.0 0.5 0.5)
    unit:
      out: (*6)
      inp: (*3 *4)
      weights: (1.0 1.5)
    unit:
      out: (*5)
      inp: (*2 *6)
      weights: (0.5 -1.0)


In [52]:
print(dumps(values, crossref=1, pretty=2, hsize=3))

network:
  nodes:
    &3 node:
      label: "x1"
    &4 node:
      label: "x2"
    &1 node:
      label: "x3"
    &2 node:
      label: "u1"
    &6 node:
      label: "u2"
    &5 node:
      label: "y"
  units:
    unit:
      out: (*2)
      inp: (*3 *4 *1)
      weights: (1.0 0.5 0.5)
    unit:
      out: (*6)
      inp: (*3 *4)
      weights: (1.0 1.5)
    unit:
      out: (*5)
      inp: (*2 *6)
      weights: (0.5 -1.0)


### Mapping named complex values to types

In [53]:
from __future__ import print_function
from axon import dump_as_str, as_unicode, factory, reduce
import numpy as np

@factory('ndarray')
def create_array(mapping, sequence):
    shape = mapping.get('shape', None)
    dtype = mapping['dtype']
    if type(dtype) is list:
        dtype = [(str(n), str(t)) for n, t in dtype]
    a = np.array(sequence, dtype=dtype)
    if shape is not None:
        a.shape = shape
    return a

@reduce(np.ndarray)
def reduce_array(a):
    signes = {'<', '=', '>', '!'}
    if len(a.dtype.descr) > 1:
        dtype = [
            (as_unicode(n), (as_unicode(t[1:]) \
                             if t[0] in signes \
                             else as_unicode(t)))
            for n, t in a.dtype.descr]
        return Node('ndarray', {'dtype':dtype}, a.tolist())
    else:
        dtype_str = a.dtype.str
        dtype_str = as_unicode(dtype_str[1:]) \
                if dtype_str[0] in signes \
                else as_unicode(dtype_str)
        return Node('ndarray', {'shape': a.shape, 'dtype':dtype_str}, a.tolist())

dump_as_str(np.int8)
dump_as_str(np.int16)
dump_as_str(np.int32)
dump_as_str(np.int64)
dump_as_str(np.float16)
dump_as_str(np.float32)
dump_as_str(np.float64)
dump_as_str(np.float128)
dump_as_str(np.int_)
dump_as_str(np.float_)
dump_as_str(np.double)


In [54]:
a = np.array([[1, 2], [3, 4], [5, 6]])
print('\nCompact form:')
text = dumps([a])
print(text)
b = loads(text, mode="strict")[0]
print('val=', repr(b))

print('\nFormatted form:')
text = dumps([a], pretty=1, hsize=3)
print(text)
print('\nIndented form:')
text = dumps([a], pretty=2, hsize=3)
print(text)
b = loads(text, mode="strict")[0]
print('val=', repr(b))

a = np.array(
        [(1, 2, 3.0), (3, 4, 5.0), (4, 5, 6.0)], 
        dtype=[('x', int), ('y', int), ('z', float)])
text = dumps([a])
print('val=', text)
b = loads(text, mode="strict")[0]
print('val=', repr(b))

text = dumps([a], pretty=1, hsize=3)
print('val=', text)
b = loads(text, mode="strict")[0]
print('val=', repr(b))
text = dumps([b], pretty=2, hsize=3)
print('val=', text)



Compact form:
ndarray{dtype:"i8" shape:(3 2) [1 2] [3 4] [5 6]}
val= array([[1, 2],
       [3, 4],
       [5, 6]])

Formatted form:
ndarray:
  dtype: "i8"
  shape: (3 2)
  [1 2]
  [3 4]
  [5 6]

Indented form:
ndarray:
  dtype: "i8"
  shape: (3 2)
  [1 2]
  [3 4]
  [5 6]
val= array([[1, 2],
       [3, 4],
       [5, 6]])
val= ndarray{dtype:[("x" "i8") ("y" "i8") ("z" "f8")] (1 2 3.0) (3 4 5.0) (4 5 6.0)}
val= array([(1, 2, 3.0), (3, 4, 5.0), (4, 5, 6.0)], 
      dtype=[('x', '<i8'), ('y', '<i8'), ('z', '<f8')])
val= ndarray:
  dtype: [
    ("x" "i8")
    ("y" "i8")
    ("z" "f8")]
  (1 2 3.0)
  (3 4 5.0)
  (4 5 6.0)
val= array([(1, 2, 3.0), (3, 4, 5.0), (4, 5, 6.0)], 
      dtype=[('x', '<i8'), ('y', '<i8'), ('z', '<f8')])
val= ndarray:
  dtype: [
    ("x" "i8")
    ("y" "i8")
    ("z" "f8")]
  (1 2 3.0)
  (3 4 5.0)
  (4 5 6.0)


## Documents

### HTML document as data

In [55]:
html_as_data = '''
html {
  head {
    meta: {charset: "utf-8"}
    title: "Sample html page"
  }
  body {

  form {
    name:"test" method:"post" action:"result.html"
    p {
      b { "Your name:" } br{}
      input { type:"text" size:"40" }
    }
    p { 
      b { "What is your browser:" } 
      input { type:"radio" name:"browser" value:"ie" } "Internet Explorer" br{}
      input { type:"radio" name:"browser" value:"opera" } "Opera" br{}
      input { type:"radio" name:"browser" value:"firefox" } "Firefox" br{}
    }
    p {
      input { type:"submit" value:"Send" }
    }
  }
  }
}
'''
values = loads(html_as_data)[0]
pprint(values)

html{head{meta:{'charset': 'utf-8'} title:'Sample html page'} body{form{name:'test' method:'post' action:'result.html'p{b{'Your name:'} br{} input{type:'text' size:'40'}} p{b{'What is your browser:'} input{type:'radio' name:'browser' value:'ie'} 'Internet Explorer' br{} input{type:'radio' name:'browser' value:'opera'} 'Opera' br{} input{type:'radio' name:'browser' value:'firefox'} 'Firefox' br{}} p{input{type:'submit' value:'Send'}}}}}


In [56]:
print(dumps(values))

head{meta:{charset:"utf-8"} title:"Sample html page"}
body{form{name:"test" method:"post" action:"result.html" p{b{"Your name:"} br{} input{type:"text" size:"40"}} p{b{"What is your browser:"} input{type:"radio" name:"browser" value:"ie"} "Internet Explorer" br{} input{type:"radio" name:"browser" value:"opera"} "Opera" br{} input{type:"radio" name:"browser" value:"firefox"} "Firefox" br{}} p{input{type:"submit" value:"Send"}}}}


In [57]:
print(dumps(values, pretty=1, hsize=4))

head:
  meta: {
    charset: "utf-8"}
  title: "Sample html page"
body:
  form:
    name: "test"
    method: "post"
    action: "result.html"
    p:
      b:
        "Your name:"
      br {}
      input:
        type: "text"
        size: "40"
    p:
      b:
        "What is your browser:"
      input:
        type: "radio"
        name: "browser"
        value: "ie"
      "Internet Explorer"
      br {}
      input:
        type: "radio"
        name: "browser"
        value: "opera"
      "Opera"
      br {}
      input:
        type: "radio"
        name: "browser"
        value: "firefox"
      "Firefox"
      br {}
    p:
      input:
        type: "submit"
        value: "Send"


In [58]:
print(dumps(values, pretty=2, hsize=4))

head:
  meta: {
    charset: "utf-8"}
  title: "Sample html page"
body:
  form:
    name: "test"
    method: "post"
    action: "result.html"
    p:
      b:
        "Your name:"
      br {}
      input:
        type: "text"
        size: "40"
    p:
      b:
        "What is your browser:"
      input:
        type: "radio"
        name: "browser"
        value: "ie"
      "Internet Explorer"
      br {}
      input:
        type: "radio"
        name: "browser"
        value: "opera"
      "Opera"
      br {}
      input:
        type: "radio"
        name: "browser"
        value: "firefox"
      "Firefox"
      br {}
    p:
      input:
        type: "submit"
        value: "Send"


### SVG example

In [59]:
text = '''
svg:
  xmlns:"http://www.w3.org/2000/svg" 
  viewBox:(0 0 300 150)
  defs:
    radialGradient:
        id: "gradient"
        cx: "50%" cy: "50%"
        stop:
          offset:"0%" 
          style:
            stop_color: rgb{200 200 200} 
            stop_opacity:0
        stop:
          offset:"100%"
          style:
            stop_color: rgb{0 0 255}
            stop_opacity:1
  ellipse:
    cx:100 cy:50 rx:100 ry:50
    style:
        fill: url{"#gradient"}
  path:
    d: "M150 0 L75 200 L225 200 Z"
  g:
    transform: [
        translate{90 30}
        rotate{-30 10}
        scale{0.9}]
  text:
    x:0 y:30 font_size:"18px" font_weight:"bold"
    text_anchor:"middle"
    "Button"
'''
values = loads(text)
pprint(values)


[svg{xmlns:'http://www.w3.org/2000/svg' viewBox:(0, 0, 300, 150)defs{radialGradient{id:'gradient' cx:'50%' cy:'50%'stop{offset:'0%'style{stop_color:rgb{200 200 200} stop_opacity:0}} stop{offset:'100%'style{stop_color:rgb{0 0 255} stop_opacity:1}}}} ellipse{cx:100 cy:50 rx:100 ry:50style{fill:url{'#gradient'}}} path{d:'M150 0 L75 200 L225 200 Z'} g{transform:[translate{90 30}, rotate{-30 10}, scale{0.9}]} text{x:0 y:30 font_size:'18px' font_weight:'bold' text_anchor:'middle''Button'}}]


In [60]:
print(dumps(values))

svg{xmlns:"http://www.w3.org/2000/svg" viewBox:(0 0 300 150) defs{radialGradient{id:"gradient" cx:"50%" cy:"50%" stop{offset:"0%" style{stop_color:rgb{200 200 200} stop_opacity:0}} stop{offset:"100%" style{stop_color:rgb{0 0 255} stop_opacity:1}}}} ellipse{cx:100 cy:50 rx:100 ry:50 style{fill:url{"#gradient"}}} path{d:"M150 0 L75 200 L225 200 Z"} g{transform:[translate{90 30} rotate{-30 10} scale{0.9}]} text{x:0 y:30 font_size:"18px" font_weight:"bold" text_anchor:"middle" "Button"}}


In [61]:
print(dumps(values, pretty=1, hsize=2))

svg:
  xmlns: "http://www.w3.org/2000/svg"
  viewBox: (
    0
    0
    300
    150)
  defs:
    radialGradient:
      id: "gradient"
      cx: "50%"
      cy: "50%"
      stop:
        offset: "0%"
        style:
          stop_color: rgb:
            200 200
            200
          stop_opacity: 0
      stop:
        offset: "100%"
        style:
          stop_color: rgb:
            0 0
            255
          stop_opacity: 1
  ellipse:
    cx: 100
    cy: 50
    rx: 100
    ry: 50
    style:
      fill: url:
        "#gradient"
  path:
    d: "M150 0 L75 200 L225 200 Z"
  g:
    transform: [
      translate:
        90 30
      rotate:
        -30 10
      scale:
        0.9]
  text:
    x: 0
    y: 30
    font_size: "18px"
    font_weight: "bold"
    text_anchor: "middle"
    "Button"


In [62]:
print(dumps(values, pretty=2, hsize=2))

svg:
  xmlns: "http://www.w3.org/2000/svg"
  viewBox: (
    0
    0
    300
    150)
  defs:
    radialGradient:
      id: "gradient"
      cx: "50%"
      cy: "50%"
      stop:
        offset: "0%"
        style:
          stop_color: rgb:
            200 200
            200
          stop_opacity: 0
      stop:
        offset: "100%"
        style:
          stop_color: rgb:
            0 0
            255
          stop_opacity: 1
  ellipse:
    cx: 100
    cy: 50
    rx: 100
    ry: 50
    style:
      fill: url:
        "#gradient"
  path:
    d: "M150 0 L75 200 L225 200 Z"
  g:
    transform: [
      translate:
        90 30
      rotate:
        -30 10
      scale:
        0.9]
  text:
    x: 0
    y: 30
    font_size: "18px"
    font_weight: "bold"
    text_anchor: "middle"
    "Button"


## Element tree

In [63]:
from xml.etree import ElementTree, cElementTree

Let's define function for dumping ``ElementTree`` structures according to ``AXON`` rules.

In [64]:
@reduce(cElementTree.Element)
@reduce(ElementTree.Element)
def element_reduce(elem):
    children = elem.getchildren()
    if elem.attrib:
        if children:
            return Node(elem.tag, elem.attrib, children)
        else:
            return Node(elem.tag, elem.attrib)
    elif children:
            return Node(elem.tag, None, children)
    else:
        return Node(elem.tag)
        
@reduce(cElementTree.ElementTree)
@reduce(ElementTree.ElementTree)
def etree_reduce(element):
    return element_reduce(element.getroot())
    
    

In [65]:
root = ElementTree.Element('persons')
ElementTree.SubElement(root, 'person', {'name':'Alice', 'age':25})
ElementTree.SubElement(root,'person', {'name':'Bob', 'age':33})
etree = ElementTree.ElementTree(root)

text = dumps([etree], pretty=1)
print(text)

persons:
  person:
    name: "Alice"
    age: 25
  person:
    name: "Bob"
    age: 33


In [66]:
def update_attribs(d):
    for key, val in d.items():
        d[key] = str(val)

class ElementTreeBuilder(Builder):
    def node(self, name, attrs, vals):
        if attrs:
            attrs = dict(attrs)
            update_attribs(attrs)
        else:
            attrs = {}
        e = cElementTree.Element(name, attrs)
        if vals:
            e.extend(vals)
        return e

register_builder('etree', ElementTreeBuilder())    

In [67]:
vals = loads(text, mode='etree')

In [68]:
cElementTree.dump(vals[0])

<persons><person age="25" name="Alice" /><person age="33" name="Bob" /></persons>
